# Task 1: Data Preparation

In [16]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('hotel_bookings.csv')
data.head()


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0.0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,1/7/2015
1,Resort Hotel,0.0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,1/7/2015
2,Resort Hotel,0.0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2/7/2015
3,Resort Hotel,0.0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2/7/2015
4,Resort Hotel,0.0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,3/7/2015


In [3]:
data.shape

(119392, 32)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119392 entries, 0 to 119391
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119392 non-null  object 
 1   is_canceled                     119391 non-null  float64
 2   lead_time                       119392 non-null  int64  
 3   arrival_date_year               119392 non-null  int64  
 4   arrival_date_month              119391 non-null  object 
 5   arrival_date_week_number        119392 non-null  int64  
 6   arrival_date_day_of_month       119392 non-null  int64  
 7   stays_in_weekend_nights         119392 non-null  int64  
 8   stays_in_week_nights            119392 non-null  int64  
 9   adults                          119392 non-null  int64  
 10  children                        119388 non-null  float64
 11  babies                          119392 non-null  int64  
 12  meal            

In [6]:
data.isna().sum()

hotel                                  0
is_canceled                            1
lead_time                              0
arrival_date_year                      0
arrival_date_month                     1
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              490
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

### Observed that agent and company has the most na values.
### dropping rows with na values in agent or company would impact the quantity of data at a scale so we just fill the agent and company with 0.

In [23]:
data['agent'] = data['agent'].fillna(0)
data['company'] = data['company'].fillna(0)

### For children columns we are going to fill na values with 0 assuming no children were there with them.

In [34]:
data['children'] = data['children'].fillna(0)

### Country of customer would be proved as an important feature of dataset making assumptions on this feature would hinder data integrity so Dropping rows with no country data 

In [41]:
data.dropna(subset=['country'], inplace=True)

### For is_cancelled column we can refer to reservation_status as observed below and fill na value with corrosponding status where reservation_status is canceled or no show it sums up to is_canceled  = 1(YES) and checkout sum is nearly equal to is_canceled = 0(NO)


In [ ]:
data['reservation_status'].value_counts()

reservation_status
Check-Out    74745
Canceled     42954
No-Show       1203
Name: count, dtype: int64

In [47]:
data['is_canceled'].value_counts()

is_canceled
0.0    74744
1.0    44157
Name: count, dtype: int64

In [49]:
data['is_canceled']=data['is_canceled'].fillna(0)

### For arrival date month we can refer to arrival week number because we can calculate which month that week number lies

### Because it is only 1 row so we are going to manually fill the value calculated for that month

In [70]:
nan_row = data[data['arrival_date_month'].isna()].index
year = data.loc[nan_row[0],'arrival_date_year']
week_num = data.loc[nan_row[0],'arrival_date_week_number']
date_num = data.loc[nan_row[0],'arrival_date_day_of_month']
pd.to_datetime(f'{year}-W{week_num}-1', format='%Y-W%W-%w').month_name()

'September'

In [71]:
data['arrival_date_month'] = data["arrival_date_month"].fillna('September')

In [77]:
data.isna().any()

hotel                             False
is_canceled                       False
lead_time                         False
arrival_date_year                 False
arrival_date_month                False
arrival_date_week_number          False
arrival_date_day_of_month         False
stays_in_weekend_nights           False
stays_in_week_nights              False
adults                            False
children                          False
babies                            False
meal                              False
country                           False
market_segment                    False
distribution_channel              False
is_repeated_guest                 False
previous_cancellations            False
previous_bookings_not_canceled    False
reserved_room_type                False
assigned_room_type                False
booking_changes                   False
deposit_type                      False
agent                             False
company                           False


## Dropping duplicated rows

In [74]:
data.duplicated().sum()

np.int64(31956)

In [76]:
data = data.drop_duplicates()